In [23]:
# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>#notebook-container { margin-left:-14px; width:calc(100% + 27px) !important; }</style>"))

In [24]:
import json, os, re, requests, subprocess

if not os.path.exists('python-utils'):
    subprocess.check_output('git clone https://github.com/CMU-CREATE-Lab/python-utils.git', shell=True)

def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']
    exec('\n'.join(src), globals())

exec_ipynb('python-utils/utils.ipynb')
exec_ipynb('python-utils/esdr-library.ipynb')
notebook_wide_display()

In [25]:
import html


In [2]:
import csv, datetime, fcntl, glob, json, math, os, re, subprocess, sys, time, xml.dom.minidom
from dateutil import rrule
from dateutil import tz
import dateutil.parser

# To install dateutil on Ubuntu
# sudo pip install python-dateutil

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()



SyntaxError: invalid syntax (<ipython-input-2-5b0f7fcb4027>, line 14)

In [8]:
# First time uploading, create a new client like so:

# Esdr.save_client('esdr-auth-purpleair-uploader.json', 'PurpleAir uploader for timemachine1')

# and then follow the directions it prints, which include visiting esdr.cmucreatelab.org and creating
# a client with given parameters, and also editing esdr-auth-baaqm-uploader.json to include your
# username and password

# Do not add esdr-auth-*.json to the git repo
# !echo 'esdr-auth-*.json' >>.gitignore

esdr = Esdr('esdr-auth-purpleair-uploader.json')
product = esdr.get_or_create_product('PurpleAir')

### Options for pulling data from PurpleAir

All sensors, current values: https://www.purpleair.com/json 




In [10]:
stationFeedCache= {}

def getStationFeed(stationId, name, lat, lon):
    if stationId in stationFeedCache:
        return stationFeedCache[stationId]
    device = esdr.get_device_by_serial_number(product, stationId)
    if not device:
        esdr.create_device(product, stationId, name=name)
        device = esdr.get_device_by_serial_number(product, stationId)

    feed = esdr.get_feed(device)
    if not feed:
        esdr.create_feed(device, lat=lat, lon=lon)
        feed = esdr.get_feed(device)
    stationFeedCache[stationId] = feed
    return feed



In [81]:
d = pd.DataFrame({2000:{'a':3, 'b':4},1000:{'a':33.5}}).transpose().sort_index()
d

,a,b
1000,33.5,NaN
2000,3.0,4.0


In [79]:
d.to_json(orient='values')

'[[3.0,4.0],[33.5,null]]'

In [76]:
d

,1000,2000
a,3,33.5
b,4,NaN


In [58]:
def upload_device(device):
    try:
        lat = float(device['Lat'])
        lon = float(device['Lon'])
    except:
        # Ignore if no parsable lat and lon
        return

    id = "%s_%06d%s%06d%s" % (device['ID'], 
                              round(1000 * abs(lat)), 'NS'[lat < 0], 
                              round(1000 * abs(lon)), 'EW'[lon < 0])
    id = id.replace('.','_')

    data = {}

    for key in ['PM2_5Value', 'RSSI', 'Uptime', 'humidity', 'pressure', 'temp_f']:
        translated_key = key
        if key == 'PM2_5Value':
            translated_key = 'PM2_5'
        try:
            data[translated_key] = float(device[key])
        except:
            pass

    try:
        stats = json.loads(device['Stats'])
        for key in stats.keys():
            if key == 'lastModified' or key == 'timeSinceModified':
                continue
            data['stats_' + key] = stats[key]
    except:
        # Stats stopped being reported Jan 2018
        pass

    try:
        data['time'] = stats['lastModified'] / 1000.0
    except:
        # Skip if no time recorded
        return

    # Move time to the beginning since that's how ESDR likes it
    keys = ['time'] + sorted(set(data.keys()).difference(set(['time'])))
    values = [data[key] for key in keys]

    # TIME is implicit for ESDR;  don't list in channel_names
    esdr_upload = {'channel_names':keys[1:], 'data':[values]}

    name = device['Label']
    feed = getStationFeed(id, name, lat, lon)
    esdr.upload(feed, esdr_upload)
    #print 'Uploaded to %s (%s)' % (id, name)

while True:
    while True:
        url = 'https://www.purpleair.com/json'
        dest_path = datetime.datetime.now().strftime('mirror/%Y%m%d/%H%M%S.json')
        response = requests.get(url, timeout=120)
        if response.status_code == 200:
            try:
                js = response.json()
                devices = js['results']
                break
            except Exception as ex:
                Stat.warning('Exception %s parsing result from %s' % (ex, url))
        else:
            Stat.warning('Response code %d from %s: %s' % (response.status_code, url, html.escape(response.text)))
        time.sleep(20)
    
    os.makedirs(os.path.dirname(dest_path), exist_ok=True)
    tmp_path = '%s-%d.tmp' % (dest_path, os.getpid())
    open(tmp_path, 'wb').write(response.content)
    os.rename(tmp_path, dest_path)
    Stat.up('Received and wrote %d devices (%.1f KB) to %s' % (len(devices), len(response.content)/1e3, dest_path),
            valid_for_secs=5*60)
    sleep_until_next_period(60)

Stat.log up Mirror purpleair realtime hal21 Received and wrote 11173 devices (6222.9 KB) to mirror/20191015/191031.json None
sleep_until_next_period(60, 0) sleeping 27 seconds until 19:11:00
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 1387 milli seconds.&quot;} None
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11173 devices (6223.1 KB) to mirror/20191015/191120.json None
sleep_until_next_period(60, 0) sleeping 35 seconds until 19:12:00
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11173 devices (6223.1 KB) to mirror/20191015/191200.json None
sleep_until_next_period(60, 0) sleeping 57 seconds until 19:13:00
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11173 devices (6223.3 KB) to mirror/20191015/191300.json None
sleep_until_next_period(60, 0) sleeping 58 seconds until 19:14:00
Stat.log up Mirro

Stat.log up Mirror purpleair realtime hal21 Received and wrote 11171 devices (6221.7 KB) to mirror/20191015/193150.json None
sleep_until_next_period(60, 0) sleeping 6 seconds until 19:32:00
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded.&quot;} None
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11171 devices (6221.7 KB) to mirror/20191015/193230.json None
sleep_until_next_period(60, 0) sleeping 27 seconds until 19:33:00
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 587 milli seconds.&quot;} None
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11171 devices (6221.7 KB) to mirror/20191015/193320.json None
sleep_until_next_period(60, 0) sleeping 37 seconds until 19:34:00
Stat.log up Mirror purpl

Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 20908 milli seconds.&quot;} None
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 592 milli seconds.&quot;} None
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 19968 milli seconds.&quot;} None
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11173 devices (6223.0 KB) to mirror/20191015/195200.json None
sleep_until_next_period(60, 0) sleeping 56 seconds until 19:53:00
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quo

Stat.log up Mirror purpleair realtime hal21 Received and wrote 11173 devices (6223.0 KB) to mirror/20191015/200750.json None
sleep_until_next_period(60, 0) sleeping 6 seconds until 20:08:00
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 20970 milli seconds.&quot;} None
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 662 milli seconds.&quot;} None
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 19631 milli seconds.&quot;} None
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11171 devices (6221.9 KB) to mirror/20191015/200901.json None
sleep_until_next_period(60,

Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 19621 milli seconds.&quot;} None
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11171 devices (6221.6 KB) to mirror/20191015/202551.json None
sleep_until_next_period(60, 0) sleeping 6 seconds until 20:26:00
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 21071 milli seconds.&quot;} None
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 722 milli seconds.&quot;} None
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot

Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 7407 milli seconds.&quot;} None
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11171 devices (6222.3 KB) to mirror/20191015/204342.json None
sleep_until_next_period(60, 0) sleeping 15 seconds until 20:44:00
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 12943 milli seconds.&quot;} None
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11171 devices (6222.2 KB) to mirror/20191015/204420.json None
sleep_until_next_period(60, 0) sleeping 37 seconds until 20:45:00
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11171 devices (6222.2 KB) to mirror/20191015/204500.json None
sleep_until_next_period(60, 0) sleeping 58 seconds unt

Stat.log up Mirror purpleair realtime hal21 Received and wrote 11171 devices (6223.0 KB) to mirror/20191015/210400.json None
sleep_until_next_period(60, 0) sleeping 58 seconds until 21:05:00
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded.&quot;} None
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 19283 milli seconds.&quot;} None
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11171 devices (6223.1 KB) to mirror/20191015/210551.json None
sleep_until_next_period(60, 0) sleeping 6 seconds until 21:06:00
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded.&quot;} None
Stat.log up Mirror purpleai

Stat.log up Mirror purpleair realtime hal21 Received and wrote 11171 devices (6223.8 KB) to mirror/20191015/211930.json None
sleep_until_next_period(60, 0) sleeping 20 seconds until 21:20:00
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 963 milli seconds.&quot;} None
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 10704 milli seconds.&quot;} None
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11171 devices (6224.0 KB) to mirror/20191015/212040.json None
sleep_until_next_period(60, 0) sleeping 18 seconds until 21:21:00
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded.&quot;}

Stat.log up Mirror purpleair realtime hal21 Received and wrote 11173 devices (6225.4 KB) to mirror/20191015/213620.json None
sleep_until_next_period(60, 0) sleeping 37 seconds until 21:37:00
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11173 devices (6225.4 KB) to mirror/20191015/213700.json None
sleep_until_next_period(60, 0) sleeping 58 seconds until 21:38:00
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded.&quot;} None
Stat.log warning Mirror purpleair realtime hal21 Response code 429 from https://www.purpleair.com/json: {&quot;code&quot;:429, &quot;message&quot;:&quot;Rate limit exceeded. Try again in 19504 milli seconds.&quot;} None
Stat.log up Mirror purpleair realtime hal21 Received and wrote 11173 devices (6225.6 KB) to mirror/20191015/213851.json None
sleep_until_next_period(60, 0) sleeping 6 seconds until 21:39:00
Stat.log warning Mirro

KeyboardInterrupt: 

In [41]:
    #dest = 
    
    # Surprisingly, the JSON is in latin1 rather than utf-8
    #decoded = body.decode('latin1')
    #js = json.loads(decoded)
    #for device in devices:
    #    upload_device(device)
    #print('Uploaded %d devices' % len(devices))

Stat.log info Mirror purpleair realtime hal21 Received 11171 devices (6221828 bytes) from https://www.purpleair.com/json None
sleep_until_next_period(60, 0) sleeping 47 seconds until 18:58:00


KeyboardInterrupt: 

In [53]:
os.path.dirname('a/b/c.d')

'a/b'

In [28]:
Stat.set_service('Mirror purpleair realtime')

In [42]:
6 * 24*60

8640

In [48]:
!ls

aggregate_data_purpleair-0.3.ipynb
aggregate_data_purpleair-2.5-thinkspeak.ipynb
aggregate_data_purpleair.ipynb
aggregate_data_purpleair.ipynb.lock
aggregate_data_purpleair.ipynb.log
aggregate-purpleair
aggregate-purpleair.ipynb
aggregates
download-parse-upload-purpleair-to-esdr.ipynb
download-parse-upload-purpleair-to-esdr.ipynb.lock
download-parse-upload-purpleair-to-esdr.ipynb.log
download-parse-upload-purpleair-to-esdr-new.ipynb
esdr-auth-purpleair-uploader.json
esdr-auth-purpleair-uploader.json~
foo0-esdr-auth-purpleair-uploader.json
mirror-purpleair-to-esdr
python-utils
python-utils-old
README.md
Untitled.ipynb


In [52]:
!ls ../mirror-ramp-to-esdr

aggregates				  ramp-v4-archive2esdr.ipynb
archive-from-aliaksei			  ramp-v4-gdrive2esdr.ipynb
config					  ramp-v4-gdrive2esdr.ipynb.lock
datadog-test.ipynb			  ramp-v4-gdrive2esdr.ipynb.log
download-parse-upload-ramp-to-esdr.ipynb  README.md
esdr-auth-ramp-uploader.json		  restart.py
gdrive					  status.py
python-utils				  stop.py
